A multiple linear regression algorithm will be employed here to attempt accurate prediction of ten year CHD based on 15 health attributes.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

In [2]:
# In order to incorporate continuous values, the uncleaned dataset is imported. 
df = pd.read_csv('framingham.csv')
df.head(10)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
5,0,43,2.0,0,0.0,0.0,0,1,0,228.0,180.0,110.0,30.30,77.0,99.0,0
6,0,63,1.0,0,0.0,0.0,0,0,0,205.0,138.0,71.0,33.11,60.0,85.0,1
7,0,45,2.0,1,20.0,0.0,0,0,0,313.0,100.0,71.0,21.68,79.0,78.0,0
8,1,52,1.0,0,0.0,0.0,0,1,0,260.0,141.5,89.0,26.36,76.0,79.0,0
9,1,43,1.0,1,30.0,0.0,0,1,0,225.0,162.0,107.0,23.61,93.0,88.0,0


In an attempt to improve prediction accuracy over the poor power of naive Bayes in this application, a multiple linear regression approach is used. The raw dataset is imported in order to make use of the continuous variables since binning is not necessary nor beneficial. 
The only form of data cleaning that will be necessary here is a simple missing value replacement that was covered in the data_cleaning file.

In [281]:
# Replacement of NA values with attribute means. 
# Dataframe copies increase memory usage but allow for checkpoints.
size = len(df)

# Attributes that will have their NA values replaced by the attribute mean. 
na_cols_mean_replace = ['cigsPerDay', 'totChol', 'BMI', 'heartRate', 'glucose']

for col in na_cols_mean_replace:
    avg = round(df[col].mean(), 2)
    df[col] = df[col].fillna(avg)

# Education will be binned as is, so replacement values will need to correspond to an existing int. 
na_cols_mean_replace_round = ['education', 'BPMeds']

for col in na_cols_mean_replace_round:
    avg = round(df[col].mean(), 0)
    df[col] = df[col].fillna(avg)


In [282]:
# Assessment of key attributes of the dataset.
def attributes(df):
    attrs = pd.DataFrame(columns=['N/A Count', 'Mean', 'Median', 'Min', 'Max'])
    for col in df:
        na_count = df[col].isna().sum()
        avg = round(df[col].mean(), 2)
        med = round(df[col].median(), 2)
        mn = min(df[col])
        mx = max(df[col])
        attrs.loc[col] = [na_count, avg, med, mn, mx]
    return attrs
    
attributes(df)

,N/A Count,Mean,Median,Min,Max
male,0.0,0.43,0.00,0.00,1.0
age,0.0,49.58,49.00,32.00,70.0
education,0.0,1.98,2.00,1.00,4.0
currentSmoker,0.0,0.49,0.00,0.00,1.0
cigsPerDay,0.0,9.00,0.00,0.00,70.0
BPMeds,0.0,0.03,0.00,0.00,1.0
prevalentStroke,0.0,0.01,0.00,0.00,1.0
prevalentHyp,0.0,0.31,0.00,0.00,1.0
diabetes,0.0,0.03,0.00,0.00,1.0
totChol,0.0,236.72,234.00,107.00,696.0


In [283]:
# Defining the independent and dependent variables. 
x = df.drop('TenYearCHD', axis=1)
y = df['TenYearCHD']

In [284]:
# Splitting the data into train and test subsets. The shape verifies the expected outcomes. 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)
print(x_train.shape, y_train.shape)

(3390, 15) (3390,)


In [285]:
# Defining and fitting the model. 
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [286]:
# Assessment of accuracy on data lacking outcome parity. 
predictions = model.predict(x_test)
predictions = (np.rint(predictions)).astype(int)

accuracy = metrics.accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
print("accuracy: {} \nF1 score: {}".format(accuracy, f1))

accuracy: 0.8679245283018868 
F1 score: 0.034482758620689655


This looks like a decent prediction accuracy score until it is considered that the base rate of CHD in the data set is rather low. In fact, a prediction model that simply predicted that every outcome was CHD-negative would attain an accuracy of roughly 85%, since only ~15% of the entries in the dataset are CHD-positive. 

The poor suitability of this test is further evinced by the very low F1 score. 

The dataset will balanced in order to rectify this issue and see if any significant improvement in accuracy can be attained.

In [287]:
# Here, enough negative CHD values are dropped until the rate of CHD is at 50%.

df_p = df.copy()

# The rows are first shuffled to remove any kind of trends in ordering.
df_p = df_p.sample(frac = 1, random_state=10)

# Dropping 2950 CHD-positive rows brings both sample sizes to 644.
df_p2 = df_p.drop(df_p[df_p.TenYearCHD==0].index[:2950])

# Percentage of positive CHD cases in the dataset.
sum(df_p2['TenYearCHD'])/len(df_p2)
#print(sum(df_p2['male']==0))

0.5

In [288]:
# Defining new variables with the parity dataset and splitting them into train and test subsets. 
x2 = df_p2.drop('TenYearCHD', axis=1)
y2 = df_p2['TenYearCHD']

x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y2, test_size=0.2, random_state=10)
print(x2_train.shape, y2_train.shape)

(1030, 15) (1030,)


In [289]:
# Fitting a new model to the parity variables. 
model2 = LinearRegression()
model2.fit(x2_train, y2_train)

LinearRegression()

In [290]:
# Getting the predicted values from the x test data via the trained model.
predictions2 = model2.predict(x2_test)

# Rounding the predicted values to the neartest int.
predictions2 = (np.rint(predictions2)).astype(int)

# Comparing the predicted values to the actual y values. 
accuracy2 = metrics.accuracy_score(y2_test, predictions2)

f1_2 = f1_score(y2_test, predictions2)

print("accuracy: {} \nF1 score: {}".format(accuracy2, f1_2))

accuracy: 0.6782945736434108 
F1 score: 0.6937269372693726


On a dataset in which outcome parity was enabled, a prediction accuracy of ~68% was achieved. The F1 score saw a massive improvement from ~0.03 to 0.69.

Since there are at least two sources of randomness involved with this training and accuracy assessment (shuffling prior to removing excess CHD-positive values and random splitting of the train and test data subsets), these sources will be run a large number of times in order to find an average. 

A k-folds cross-analysis technique would be standard practice to use in alleviating some of the randomness issues in splitting data into train and test data. There is a built in scikit function for this; however, it is not useful given that the prediction model outputs floats rather than the ints that are expected in the outcome. 

The weakness of the k-folds function is demonstrated below, followed by a tailored implementation of a similar function designed to minimize the impact of randomness. 

In [291]:
# A k-folds cross-analysis. Doesn't work well since the resulting predictions are floats rather than the expected binary. 

mod = LinearRegression()
scores = cross_val_score(mod, x2, y2, cv=5)
scores.mean()


-0.5319685507018118

In [315]:
# Function that determines average model accuracy and F1 score based on different random train/test splits. 
# Similar to a k-folds cross-analysis, but allows for more randomization of data and performs a critical rounding step.


def k_folds(n, x, y):
    accuracies = []
    f1s = []
    for k in range(1, n+1):
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=k)
    
        model = LinearRegression()
        model.fit(x_train, y_train)
    
        predictions = model.predict(x_test)
        predictions = (np.rint(predictions)).astype(int)

        accuracy = metrics.accuracy_score(y_test, predictions)
        f1 = f1_score(y_test, predictions, average='macro')
    
        accuracies.append(accuracy)
        f1s.append(f1)
    
    return (np.mean(accuracies), np.mean(f1s))


# Sample output of 10 random splits. 
k_folds(100, x2, y2)


(0.6675193798449613, 0.6603840264659029)

The remaining source of randomness is due to the shuffling of the dataset before removing excess CHD-positive values. To get a good measure of training accuracy, a number 'n' of random shufflings will be performed, followed by a number of randomized cross-analysis previously implemented. 

In [316]:
# Now testing different removals of CHD-negative rows that were taken out to achieve parity.

n = 100
total_accuracies = []
total_f1s = []

for x in range(1,n+1):
    df_r = df.copy()
    
    # The rows are first shuffled to remove any kind of trends in ordering.
    df_r = df_r.sample(frac = 1, random_state=x)

    df_r2 = df_r.drop(df_r[df_r.TenYearCHD==0].index[:2950])
    
    xn = df_r2.drop('TenYearCHD', axis=1)
    yn = df_r2['TenYearCHD']
    
    
    accuracy_of_shuffle = k_folds(x, xn, yn)
    
    total_accuracies.append(accuracy_of_shuffle[0])
    total_f1s.append(accuracy_of_shuffle[1])
    

print("accuracy: {} \nF1 score: {}".format(np.mean(total_accuracies), np.mean(total_f1s)))


accuracy: 0.6646889986077095 
F1 score: 0.6628178675107267


100 random shufflings, each resulting in 100 random cross-analyses, resulted in a overall mean prediction accuracy of ~66%. This is much better than the previously obtained naive Bayes prediction accuracy of ~57%. Additionally, the mean F1 score of ~0.66 is a large improvement over the data that was trained without regard to parity. 

Given that medical conditions tend to occur with a high amount of variance (cases in which individuals with low risk factors are often diagnosed with the disease while individuals with many risk factors avoid the disease befalling them), it is quite possible that a high prediction accuracy is possible. 

Overall, I believe the prediction accuracy of ~66% and F1 score of 0.66 to be satisfactory in this case, though there is almost certainly room for future improvement. 